# Notebook to realize Non-target analysis for FF and Nurses
Set of scripts to realize the Non target analysis with the Women Worker Biomonitoring Chemical Database (WWDB) for nurses and Firefighter

In [1]:
from os import path

# probably need to import in step not globally
from prepForFrag import prepForFrag
from WWBC_database import WWBC_database, matchChemicals
from toolbox import pathFolder, toolbox
from filterAnnotation import filterAnnotation
from mapAfterFrag import mapAfterFrag
from checkPoint import checkDB, checkTable
from confirmFrag import confirmFrag

# define path #
#################
PR_ROOT = path.abspath("../../") + "/"
PR_DATA = PR_ROOT + "data/"
PR_RESULTS = pathFolder.createFolder(PR_ROOT + "results/")

## 1. Step1: Women Worker Biomonitoring Chemical Database Development (v.2)

### 1.1. Initialization

In [2]:
# SET CRITERIA #
################
minMW = 100
maxMW = 1000
lipinski_violation = 3
list_chemicals_metabolite = ["Drug_UCSF_PXYS", "Drug_most comon and haz", "Disinfectant", "FRs", "PFAS", "MC", "MGDev", "ERactive_bin", "E2Up_bin", "P4Up_bin", "pesticidemammarytumors_bin", "nitroPAH_bin", "pellizzari_bin", "phthalate"]

# SET FILES USE IN INPUT #
##########################
p_UCSF_Haz_Drugs_Inventory = PR_DATA + "buildDB/UCSF_Haz_Drugs Inventory_FINAL 2_2018_wDTSXIDs.csv"
p_drug_in_survey = PR_DATA + "buildDB/nurses_otherdrugs_rar_10272021.csv"
p_WWBC_v1 = PR_DATA + "buildDB/WWBC_MS_database_10.28.21.csv"

### 1.2. Prepare WWBC

In [4]:
# RUN - PREPARE WWBC database #
###############################
l_d_fileToMap = [{"pfile":p_UCSF_Haz_Drugs_Inventory, "headertoMap": "DTXSID (CompTox)", "addCol": "UCSF_haz_drug"}, {"pfile":PR_DATA + p_drug_in_survey, "headertoMap":"DTSXID (from wwbc database plus other sources)", "addCol": "Survey_drug"}]
name_DB = "WWBC_MS_database_10.28.21"
c_db = WWBC_database.WWBC_database(p_WWBC_v1, minMW, maxMW, lipinski_violation, list_chemicals_metabolite, PR_RESULTS, l_d_fileToMap, name_DB)
p_WWBC_v2 = c_db.main()

print("WWDB ready for mapping")
print("Path:", p_WWBC_v2)

WWDB ready for mapping
Path: /mnt/c/Users/AlexandreBorrel/research/SSI/NTA/results/WWBC_MS_database_10.28.21_prepForAnotation.csv


### 1.3. Check WWBC

In [ ]:
# CHECK WWBC - control if some chemicals are properly included in the WWWBC #
############################################################################# 

# create folder to check
pr_checkpoint = pathFolder.createFolder(PR_RESULTS + "CHECKPOINT_step1_WWBC/")
#p_out = "/mnt/c/Users/AlexandreBorrel/research/SSI/NTA/results/check/chem_in_DB.txt"

l_DTXID_tocheck = ["DTXSID1022556", "DTXSID9022601","DTXSID9020116","DTXSID3020910","DTXSID3046742","DTXSID4024983","DTXSID1022845","DTXSID5020364","DTXSID0020365","DTXSID3022877","DTXSID5020364","DTXSID7022883","DTXSID70227388","DTXSID2021735","DTXSID8021480","DTXSID1020562","DTXSID5023035","DTXSID9023049","DTXSID3020625","DTXSID4039657","DTXSID2020634","DTXSID9044299","DTXSID6020648","DTXSID8041032","DTXSID4034150","DTXSID4025371","DTXSID6025438","DTXSID7020760","DTXSID9037664","DTXSID6020804","DTXSID4020822","DTXSID2020892","DTXSID00858942","DTXSID4041070","DTXSID0045703","DTXSID9023413","DTXSID7023431","DTXSID8020541","DTXSID8023135","DTXSID4021185","DTXSID8023557","DTXSID5040910","DTXSID6034186","DTXSID5046354","DTXSID1034187","DTXSID3023631","DTXSID8022371","DTXSID8023688","DTXSID8048288","DTXSID30154863","DTXSID1032278","DTXSID5023742","DTXSID9046023"]
k_search = "DTXSID"

c_check = checkDB.checkDB(p_WWBC_v2)
c_check.searchInDB(k_search, l_DTXID_tocheck, pr_checkpoint + "chem_in_DB.txt")

## 2. Step 2: Non-targeted LC-QTOF/MSfull scan analysis procedure (and developing the features list)

In [ ]:
### DO not know exactly where it is going -- maybe it is a checking

# MAP NURSE RESULTS ON MODE FROM MS
p_filin = PR_DATA + "nurse_drug_selectionRR_10-29-21/all_considered_for_target_N_10_29_21.csv"
# map mode on it
p_filout = PR_DATA + "nurse_drug_selectionRR_10-29-21/all_considered_for_target_N_10_29_21_mappedOnMode.csv"

p_pos =  PR_DATA + "nurse_drug_selectionRR_10-29-21/List_matched_no_filter_neg_FB_06.21_rar.csv"
p_neg = PR_DATA + "nurse_drug_selectionRR_10-29-21/List_matched_no_filter_pos_FB_06.21_rar.csv"


d_tomap = [{"pfile":p_pos, "headertoMap": "DB_name", "addCol": "POS mode"},{"pfile":p_neg, "headertoMap": "DB_name", "addCol": "NEG mode"} ]
toolbox.manualMapping(p_filin, d_tomap, p_filout)

## 3. Step 3: Review of features and criteria to select features for fragmentation

In this step we processed differently Nurse vs Office worker and Firefighter.

### 3.1. Initialization

In [2]:
# Define path of mapping - after XCMS processing
## NURSE
### positive mode
p_pos_nurse = PR_DATA + "result_NTA_tofilter/List_matched_no_filter_pos_N_FB_07.21_rar.csv"
### negative mode
p_neg_nurse = PR_DATA + "result_NTA_tofilter/List_matched_no_filter_neg_FB_07.21.csv"

## FF
### positive mode
p_pos_FF = PR_DATA + "result_NTA_tofilter/20210716POS_features_FF_limited_filtered.csv"
### negative mode
p_neg_FF = PR_DATA + "result_NTA_tofilter/20210716Neg_features_FF_limited_filtered.csv"



### 3.2. Fragmentation for Nurses and Office workers

### 3.3. Fragmentation for FF

In [ ]:
# step that match 
pr_out = pathFolder.createFolder(PR_RESULTS + "step3_ matchFFwithChemSubstrcuture/")
d_substructures = {"nitro benzene":["[O-][N+](=O)C1=CC=CC=C1"], "nitro":["[O-][N+](=O)C"], 
                   "sulfate":["S(=O)(=O)([O-])C"], "aniline":["Nc1ccccc1"], "guanidium":["NC(N)=N"]}

# negative mode
p_neg_match = PR_DATA + "match_chemicals/20210501list_matched_NEGfeaturesFF_Priority.xlsx"
c_match = matchChemicals.matchChemicals(p_neg_match, "NEG_match",pr_out)
c_match.loadChem("20210501list_matched_NEGfeature")
c_match.searchSubstructure(d_substructures)


# positive mode
p_pos_match = PR_DATA + "match_chemicals/20210501list_matched_POSfeatures_FF_prioritize.xlsx"
c_match = matchChemicals.matchChemicals(p_pos_match, "POS_match",pr_out)
c_match.loadChem("20210501list_matched_POSfeature")
c_match.searchSubstructure(d_substructures)

# here unique feature

### 3.4. Condensate results from OW/N and FF

#### 3.4.1. Criteria used - generalized using a txt file and 3 types of criteria
- criteria M apply on main matrix
- criteria A apply for annotation, will not impact the all matrix
- criteria D apply on the main matrix and remove un selected feature

In [3]:
## define file with criteria to apply
p_filter_criteria = PR_DATA + "result_NTA_tofilter/filter_criteria.txt"

## show criteria ##
###################
f_filter = open(p_filter_criteria, "r")
criteria = f_filter.read()
f_filter.close()

### 3.4.2. Run annotation

In [4]:
# run for the negative mode
pr_out = pathFolder.createFolder(PR_RESULTS + "forFragNeg/")
l_p_annotation = [p_neg_nurse, p_neg_FF]

c_prepForFrag = prepForFrag.prepForFrag(l_p_annotation, p_filter_criteria, "NEG", pr_out)
c_prepForFrag.verbose = 1
c_prepForFrag.filteredAnnotation()
#c_prepForFrag.mergeAnnotationForFrag()

CriteriaM1. ['percent_FB <= 50']
CriteriaA1. ['higher_intensity_N == 1 OR DF_high_N == 1 OR cv_N/cv_OW >= 1.5 OR sd_N >= AVG sd_N', 'Drug_UCSF_PXYS == 1 OR FRs == 1 OR PFAS == 1 OR Disinfectant == 1 OR E2Up_bin == 1 OR P4Up_bin == 1 OR ERactive_ER_AUC >= 0.01 OR nitroPAH_bin == 1 OR pellizzari_bin == 1 OR phthalate == 1 OR MC == 1 OR MGDev == 1']
CriteriaA2. ['higher_intensity_N == 1 OR DF_high_N == 1 OR cv_N/cv_OW >= 1.5 OR sd_N >= AVG sd_N', 'sd_N >= AVG sd_N', 'exposurepred_bin == 1']
CriteriaM1. ['percent_FB <= 50']
ERROR colname: percent_FB
CriteriaA1. ['higher_intensity_N == 1 OR DF_high_N == 1 OR cv_N/cv_OW >= 1.5 OR sd_N >= AVG sd_N', 'Drug_UCSF_PXYS == 1 OR FRs == 1 OR PFAS == 1 OR Disinfectant == 1 OR E2Up_bin == 1 OR P4Up_bin == 1 OR ERactive_ER_AUC >= 0.01 OR nitroPAH_bin == 1 OR pellizzari_bin == 1 OR phthalate == 1 OR MC == 1 OR MGDev == 1']
ERROR colname: higher_intensity_N
ERROR colname: DF_high_N
ERROR colname: sd_N
ERROR colname: phthalate
CriteriaA2. ['higher_intensi

/home/aborrel/miniconda3/envs/QSAR_modeling/lib/python3.9/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/aborrel/miniconda3/envs/QSAR_modeling/lib/python3.9/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [5]:
# run for positive mode
pr_out = pathFolder.createFolder(PR_RESULTS + "forFragPos/")
l_p_annotation = [p_pos_nurse, p_pos_FF]

c_prepForFrag = prepForFrag.prepForFrag(l_p_annotation, p_filter_criteria, "POS", pr_out)
c_prepForFrag.verbose = 1
c_prepForFrag.filteredAnnotation()
#c_prepForFrag.mergeAnnotationForFrag()

CriteriaM1. ['percent_FB <= 50']
CriteriaA1. ['higher_intensity_N == 1 OR DF_high_N == 1 OR cv_N/cv_OW >= 1.5 OR sd_N >= AVG sd_N', 'Drug_UCSF_PXYS == 1 OR FRs == 1 OR PFAS == 1 OR Disinfectant == 1 OR E2Up_bin == 1 OR P4Up_bin == 1 OR ERactive_ER_AUC >= 0.01 OR nitroPAH_bin == 1 OR pellizzari_bin == 1 OR phthalate == 1 OR MC == 1 OR MGDev == 1']
CriteriaA2. ['higher_intensity_N == 1 OR DF_high_N == 1 OR cv_N/cv_OW >= 1.5 OR sd_N >= AVG sd_N', 'sd_N >= AVG sd_N', 'exposurepred_bin == 1']
CriteriaM1. ['percent_FB <= 50']
ERROR colname: percent_FB
CriteriaA1. ['higher_intensity_N == 1 OR DF_high_N == 1 OR cv_N/cv_OW >= 1.5 OR sd_N >= AVG sd_N', 'Drug_UCSF_PXYS == 1 OR FRs == 1 OR PFAS == 1 OR Disinfectant == 1 OR E2Up_bin == 1 OR P4Up_bin == 1 OR ERactive_ER_AUC >= 0.01 OR nitroPAH_bin == 1 OR pellizzari_bin == 1 OR phthalate == 1 OR MC == 1 OR MGDev == 1']
ERROR colname: higher_intensity_N
ERROR colname: DF_high_N
ERROR colname: sd_N
ERROR colname: phthalate
CriteriaA2. ['higher_intensi

### 3.5. Checkpoint
Control the mapping

In [ ]:
#check merge for frag
# check comfirmation list
pfile1 = "/mnt/c/Users/AlexandreBorrel/research/SSI/NTA/data/merge_check_12-15-21/all_considered_for_target_N_11.8.21_LH ranking_rar_ETF merge_rar2_AB_ETF_rar_LB.csv"
pfile2 = "/mnt/c/Users/AlexandreBorrel/research/SSI/NTA/data/merge_check_12-15-21/N_FFConfirmationAndSemitarget_list.csv"
p_out = "/mnt/c/Users/AlexandreBorrel/research/SSI/NTA/data/merge_check_12-15-21/log_diff.txt"
p_pos_mode = "/mnt/c/Users/AlexandreBorrel/research/SSI/NTA/data/merge_check_12-15-21/POS_filter_features_3MW_30deltaRT_NurseFFCombined.csv"
p_neg_mode = "/mnt/c/Users/AlexandreBorrel/research/SSI/NTA/data/merge_check_12-15-21/NEG_filter_features_3MW_30deltaRT_NurseFFCombined.csv"

cdiff = checkTable.checkTable(pfile1, pfile2, p_out)
cdiff.loadFile()
cdiff.compareCol()
cdiff.diffValueInCol("DTXSID", ["X"] )#["name_original", "ID", "SMILES", "SMILES_cleaned", "Molweight_cleaned", "ID_raw"])
cdiff.checkDuplicate(pfile=2, col_header="DTXSID", l_merge_col = ["featureid.x", "featureid.y", "parent.mass", "RTMean", "MW", "RT", "revised.priority.based.on.likelihood.to.work.on.lc.ms...1st.20", "revised.priority.based.on.likelihood.to.work.on.lc.ms...2st.20", "choose.for.targeted.long.and.older.", "POS.mode",
                                                                                 "NEG.mode", "neg.pos.from.JT", "LH.rank", "LH.notes", "DTSC.and.other.chemists..any.concern.with.compatability.with.your.methods..chemical.stability.in.serum", 
                                                                                 "DTSC...has.standards.", "Elissia.ID.supplier", "Link.to.order", "QC.EPA", "In.volatile.EPA.list", "In.pubchem", "Polarizability", "LC.MS.compatible..Elissia.",
                                                                                 "X.1", "X.2", "mode", "name", "group", "mean_visit1", "mean_visit2", "mean_visit3", "mean_N", "mean_OW"], p_out = "/mnt/c/Users/AlexandreBorrel/research/SSI/NTA/data/merge_check_12-15-21/log_dup.csv")

cdiff.checkDupAndTrackFeatureOrigin(pfile=2, col_header="DTXSID", l_merge_col = ["featureid.x", "featureid.y", "parent.mass", "RTMean", "MW", "RT", "revised.priority.based.on.likelihood.to.work.on.lc.ms...1st.20", "revised.priority.based.on.likelihood.to.work.on.lc.ms...2st.20", "choose.for.targeted.long.and.older.", "POS.mode",
                                                                                 "NEG.mode", "neg.pos.from.JT", "LH.rank", "LH.notes", "DTSC.and.other.chemists..any.concern.with.compatability.with.your.methods..chemical.stability.in.serum", 
                                                                                 "DTSC...has.standards.", "Elissia.ID.supplier", "Link.to.order", "QC.EPA", "In.volatile.EPA.list", "In.pubchem", "Polarizability", "LC.MS.compatible..Elissia.",
                                                                                 "X.1", "X.2", "mode", "name", "group", "mean_visit1", "mean_visit2", "mean_visit3", "mean_N", "mean_OW"], l_features_col=["featureid.x", "featureid.y"], d_origin = {"POS":toolbox.loadMatrix(p_pos_mode), "NEG": toolbox.loadMatrix(p_neg_mode)}, p_out = "/mnt/c/Users/AlexandreBorrel/research/SSI/NTA/data/merge_check_12-15-21/log_dup_mode.csv")


cdiff.checkIfInfragList(pfile1, check_col = "DTXSID", flag_col= "choose for targeted 1st 20", d_frag = {"POS":toolbox.loadMatrix(p_pos_mode, sep = ","), "NEG": toolbox.loadMatrix(p_neg_mode, sep = ",")}, p_out = "/mnt/c/Users/AlexandreBorrel/research/SSI/NTA/data/merge_check_12-15-21/log_mode.csv")




## 4. Step 4: Fragmentation and annotation

In [ ]:
# FROM fragMatch -- need to update



# frag match positive node
#pr_out = pathFolder.createFolder(PR_RESULTS + "fragMatch/")
#p_output_frag_pos = PR_DATA + "resultFrag10-2021/node_attributes_table_pos1.tsv" #~/Box/SHE lab/WWBC/Data analysis/Non-targeted analysis/2021Fragmentation matching/node_attributes_table_pos1.tsv
#p_db_frag_pos = PR_DATA + "resultFrag10-2021/POS_filter_features_3MW_30deltaRT_NurseFFCombined.csv" #"~/Box/SHE lab/WWBC/Data analysis/Non-targeted analysis/2021Fragmentation list to DTSC/POS_filter_features_3MW_30deltaRT_NurseFFCombined.csv"
#p_original_DB = PR_RESULTS + "WWBC_MS_database_6.30.21_prepForAnotation.csv"

#fragMatch(p_output_frag_pos, p_db_frag_pos, p_original_DB, 1, "Positive", pr_out)
#fragMatch(p_output_frag_pos, p_db_frag_pos, p_original_DB, 2, "Positive", pr_out)
#fragMatch(p_output_frag_pos, p_db_frag_pos, p_original_DB, 3, "Positive", pr_out)

#p_output_frag_neg = PR_DATA + "resultFrag10-2021/node_attributes_table_neg.tsv" #~/Box/SHE lab/WWBC/Data analysis/Non-targeted analysis/2021Fragmentation matching/node_attributes_table_pos1.tsv
#p_db_frag_neg = PR_DATA + "resultFrag10-2021/NEG_filter_features_3MW_30deltaRT_NurseFFCombined.csv" #"~/Box/SHE lab/WWBC/Data analysis/Non-targeted analysis/2021Fragmentation list to DTSC/POS_filter_features_3MW_30deltaRT_NurseFFCombined.csv"

#fragMatch(p_output_frag_neg, p_db_frag_neg, p_original_DB, 1, "Negative", pr_out, 2.0)
#fragMatch(p_output_frag_neg, p_db_frag_neg, p_original_DB, 2, "Negative", pr_out, 2.0)
#fragMatch(p_output_frag_neg, p_db_frag_neg, p_original_DB, 3, "Negative", pr_out, 2.0)


# comparison output
#p_alex = pr_out + "Negative_Digit-3_deltaMass-2.0.csv"
#p_jess = pr_out + "NEG_matched_confirmed_3digits.csv"

#checkResult(p_jess, p_alex)


#p_alex = pr_out + "Negative_Digit-2_deltaMass-2.0.csv"
#p_jess = pr_out + "NEG_matched_confirmed_2digits.csv"

#checkResult(p_jess, p_alex)

In [ ]:
# rematch on DB with the non matching on fragmentation 

## 5. Step 5: Chemical selection criteria for targeted confirmation and quantification 

### 5.1. Initialization

In [1]:
from toolbox import pathFolder, toolbox
from os import path
PR_ROOT = path.abspath("../../") + "/"
PR_DATA = PR_ROOT + "data/"
PR_RESULTS = pathFolder.createFolder(PR_ROOT + "results/")
# define out put path for step 5 #
##################################
pr_data_step5 = PR_DATA + "toConformFrag_12-21-21/"

# INPUT file #
##############
p_chem_frag = pr_data_step5 + "all_considered_for_target_N_11.8.21_LH_ranking_rar_ETF_merge_rar2_AB_ETF_rar_LB.csv"#List of target chemicals (only need lines 1-42 - 1 n column H or I): https://berkeley.box.com/s/wsqe11x7de9quea15un2ta5n4bmsyval
p_nurse_pos = pr_data_step5 + "List_matched_no_filter_pos_FB_06.21_rar.csv"#Nurse pos features:https://berkeley.box.com/s/r1b5z7vs9nt8alf6s9yyw2huz1n0n9yv
p_nurse_neg = pr_data_step5 + "List_matched_no_filter_neg_FB_06.21_rar.csv"#nurse negative features: https://berkeley.box.com/s/a0h4qxl8arbn7wlel6jc92d26v3pd2hr
p_FF_neg = pr_data_step5 +  "20210716Neg_features_FF_limited.csv"#FF pos features https://berkeley.box.com/s/ot52cqpdc1di4nc01oxg9coqvwftr078
p_FF_pos = pr_data_step5 +  "20210716POS_features_FF_limited.csv"#FF neg features  https://berkeley.box.com/s/gbd7lvpl5m7mp7i3r5e8tia4y01by8pc
p_node_neg = pr_data_step5 +  "node_attributes_table_neg.tsv"
p_node_pos = pr_data_step5 +  "node_attributes_table_pos1.tsv"
p_WWBC_db = PR_RESULTS + "WWBC_MS_database_10.28.21_prepForAnotation.csv"

### 5.2. Confirmation fragmentation with mapping
In this step we will map the results of fragmentation 

In [2]:
from confirmFrag import confirmFrag
l_select_target_chem = ["revised priority based on likelihood to work on lc/ms - 1st 20", "revised priority based on likelihood to work on lc/ms - 2st 20"]
l_col_target_chem = ["revised priority based on likelihood to work on lc/ms - 1st 20", "revised priority based on likelihood to work on lc/ms - 2st 20", "CASRN","name_original","formula_original","Elissia.ID.supplier", "Link.to.order", "QC EPA", "In volatile EPA list", "In pubchem", "Polarizability", "LC-MS compatible (Elissia)", "ionization", "comments"]
d_col_study = {"nurse":["mz","time", "mean_N", "mean_OW", "mean_FB"], "FF":["mz.y","rt.y", "mean_allvisit", "mean_FB"]}

#l_select_target_chem, l_col_target_chem, l_col_analysis, p_out
p_data_confirm = pr_data_step5 + "mapping_output_1-5-22.csv"

c_confirm_frag = confirmFrag.confirmFrag(p_chem_frag, p_nurse_pos, p_nurse_neg, p_FF_pos, p_FF_neg, p_node_neg, p_node_pos, p_WWBC_db)
c_confirm_frag.loadDataSet()
c_confirm_frag.mapFeatureToChem(l_select_target_chem, l_col_target_chem, p_data_confirm)

## count by chemical classes remap on the database with dtxsid
c_confirm_frag.summarizeConfirmFrag()

Number confirm frag: 121
Number of unique dsstoxid: 41
Number of DSSTOX ID without feature ID: 16
Drug_UCSF_PXYS: 15
Drug_most comon and haz: 5
Disinfectant: 2
Sesquiterpenoids: 0
FRs: 0
PFAS: 7
exposurepred_bin: 12
MC: 4
MGDev: 0
ERactive_bin: 2
E2Up_bin: 7
E2UP_priority_level: 3
P4Up_bin: 7
P4Up_priority_level: 1
pesticidemammarytumors_bin: 0
VBtoxcast_bin: 25
nitroPAH_bin: 0
pellizzari_bin: 3
UCSF_haz_drug: 12
Survey_drug: 14
phthalate: 1
